In [26]:
from utils import *
from rich import print as rprint
import os
from numba import jit, njit
import copy
import track_utils
import statscols

In [27]:
# load h5py data
data = load_ims(
    "../../data/surface_track_parser_dev_data/ALN Sec2 Roi1 2x2 1h20min.ims"
)

In [28]:
points = track_utils.get_points(data)
rprint("points: ", points)

points: 
['MegaSurfaces0', 'MegaSurfaces1', 'MegaSurfaces2', 'MegaSurfaces3']

In [29]:
stats_names = np.asarray(data["Scene8"]["Content"]["MegaSurfaces0"]["StatisticsType"])
stats_names

array([(16759, 0, 0, b'Acceleration', b'um/s^2'),
       (16759, 0, 0, b'Acceleration', b'um/s^2'),
       (16759, 0, 0, b'Acceleration', b'um/s^2'), ...,
       (16991, 0, 0, b'Volume', b'um^3'),
       (16991, 0, 0, b'Volume', b'um^3'),
       (16991, 0, 0, b'Volume', b'um^3')],
      dtype=[('ID', '<i8'), ('ID_Category', '<i8'), ('ID_FactorList', '<i8'), ('Name', 'S256'), ('Unit', 'S256')])

In [30]:
stats_values = pd.DataFrame(
    np.asarray(data["Scene8"]["Content"]["MegaSurfaces0"]["StatisticsValue"])
)
stats_values

,ID_Time,ID_Object,ID_StatisticsType,Value
0,-1,-1,16,2.675000e+03
1,-1,-1,1828,4.595600e+04
2,-1,-1,768,4.595600e+04
3,-1,-1,1827,1.607588e+07
4,-1,-1,11260,3.436532e+06
...,...,...,...,...
6452821,45,66616,16991,1.725892e+02
6452822,45,66867,16991,2.079986e+02
6452823,45,66875,16991,8.757520e+01
6452824,45,66920,16991,1.668989e+02


In [31]:
track_ids = pd.DataFrame(
    np.asarray(data["Scene8"]["Content"]["MegaSurfaces0"]["Track0"])
)["ID"]
type(track_ids)

pandas.core.series.Series

In [32]:
# get stats dict for whole ims file
full_stats_dict = track_utils.get_statistics_dict(data, "MegaSurfaces0")
invert_stats_dict = track_utils.invert_stats_dict(full_stats_dict)

In [33]:
# filter dataset to only have tracks
track_only_df = stats_values[stats_values["ID_Object"].isin(track_ids)]

In [34]:
# filter the full stats dict to only have the info from the tracks
filtered_stats = {}
for stat_type in track_only_df["ID_StatisticsType"].to_list():
    filtered_stats[stat_type] = full_stats_dict[stat_type]

full_stats_dict

{16759: b'Acceleration',
 16760: b'Acceleration X',
 16761: b'Acceleration Y',
 16762: b'Acceleration Z',
 11263: b'Area',
 16763: b'BoundingBoxAA Length X',
 16764: b'BoundingBoxAA Length Y',
 16765: b'BoundingBoxAA Length Z',
 16766: b'BoundingBoxOO Length A',
 16767: b'BoundingBoxOO Length B',
 16768: b'BoundingBoxOO Length C',
 16769: b'Center of Homogeneous Mass X',
 16770: b'Center of Homogeneous Mass Y',
 16771: b'Center of Homogeneous Mass Z',
 16772: b'Center of Image Mass X',
 16773: b'Center of Image Mass X',
 16774: b'Center of Image Mass X',
 16775: b'Center of Image Mass X',
 16776: b'Center of Image Mass X',
 16777: b'Center of Image Mass X',
 16778: b'Center of Image Mass Y',
 16779: b'Center of Image Mass Y',
 16780: b'Center of Image Mass Y',
 16781: b'Center of Image Mass Y',
 16782: b'Center of Image Mass Y',
 16783: b'Center of Image Mass Y',
 16784: b'Center of Image Mass Z',
 16785: b'Center of Image Mass Z',
 16786: b'Center of Image Mass Z',
 16787: b'Center of

In [35]:
inv_filtered = track_utils.invert_stats_dict(filtered_stats)

In [44]:
invert_stats_dict['Time Index']

16873

In [37]:
full_stats_dict = track_utils.get_statistics_dict(data, "MegaSurfaces0")
full_stats_dict[16873]

b'Time Index'

In [40]:
f = stats_values[stats_values["ID_StatisticsType"] == 16873]
first_objects = f[f["Value"] == 1]['ID_Object']

In [43]:
stats_values[stats_values['ID_Object'].isin(first_objects)]

,ID_Time,ID_Object,ID_StatisticsType,Value
294630,0,0,16759,0.000000
294631,0,2,16759,0.000000
294632,0,8,16759,0.000000
294633,0,12,16759,0.000000
294634,0,21,16759,0.000000
...,...,...,...,...
404641,0,1429,16991,909.187439
404642,0,1434,16991,390.645935
404643,0,1435,16991,431.511841
404644,0,1437,16991,140.038055


In [20]:
track_and_object_id_info = track_utils.convert_to_matrix(
    track_id_data, track_object_data
)

In [21]:
temp_dict = track_utils.create_dict(track_and_object_id_info)

In [22]:
stats_data = track_utils.extract_data(track_and_object_id_info, stats_values)

In [23]:
stats_dict[16969]

b'Track Position X Start'

In [24]:
inv_stats_dicts = track_utils.invert_stats_dict(stats_dict)

In [32]:
inv_stats_dicts["Time"]

16872

In [68]:
statscols = statscols.StatsColumns(
    0,
    stats_dict,
    track_id_data,
    track_object_data,
    stats_values,
    track_and_object_id_info,
    stats_data,
    inv_stats_dicts,
)

AttributeError: 'StatsColumns' object has no attribute 'StatsColumns'

In [36]:
statscols

In [9]:
## can we get surfaces by modifying following function
def get_object_names(full_data_file: h5py.File, search_for: str) -> list:
    """
    _summary_

    Args:
        full_data_file (h5py._hl.files.File): full imaris file in h5py File format
        search_for (str): string containing full or partial filename to search for

    Returns:
        (list): a list of all the object names that match search_for parameter
    """

    values = full_data_file.get("Scene").get("Content").keys()
    storage = list()
    for item in values:
        if len(re.findall(search_for, item)):
            storage.append(item)
    return storage


# test function -- working
out = get_object_names(data, "Surface")
print(out)

['MegaSurfaces0', 'MegaSurfaces1', 'MegaSurfaces2', 'MegaSurfaces3']


In [9]:
# once we know the object names we want to extract data from use following function
# this function doesnt grab any data values, just names
def get_statistics_names(full_data_file: h5py.File, object_name: str) -> dict:
    """
    for a given object_name, extracts the statistics names and ids into a dict
    ex: statistics name = mean intensity, associated id=404

    Args:
        full_data_file (h5py._hl.files.File): full imaris file in h5py File format
        object_name (str): name of the object to get statistic names from

    Returns:
        dict: a dict where the keys=unique stats ID, value=static name
    """

    # get object specific data
    obj_specific_data = full_data_file["Scene8"]["Content"][object_name]

    # rearrange data
    statistics_name = np.asarray(obj_specific_data["StatisticsType"])
    statistics_name = pd.DataFrame(statistics_name)

    # extract statistics names
    stats_name = statistics_name["Name"]

    # extract statistics ID names
    stats_type = statistics_name["ID"]

    # combine stats type and stats names
    return dict(zip(stats_type, stats_name))


# test -- working
out_stats_names = get_statistics_names(data, "MegaSurfaces0")

out_stats_names

{16759: b'Acceleration',
 16760: b'Acceleration X',
 16761: b'Acceleration Y',
 16762: b'Acceleration Z',
 11263: b'Area',
 16763: b'BoundingBoxAA Length X',
 16764: b'BoundingBoxAA Length Y',
 16765: b'BoundingBoxAA Length Z',
 16766: b'BoundingBoxOO Length A',
 16767: b'BoundingBoxOO Length B',
 16768: b'BoundingBoxOO Length C',
 16769: b'Center of Homogeneous Mass X',
 16770: b'Center of Homogeneous Mass Y',
 16771: b'Center of Homogeneous Mass Z',
 16772: b'Center of Image Mass X',
 16773: b'Center of Image Mass X',
 16774: b'Center of Image Mass X',
 16775: b'Center of Image Mass X',
 16776: b'Center of Image Mass X',
 16777: b'Center of Image Mass X',
 16778: b'Center of Image Mass Y',
 16779: b'Center of Image Mass Y',
 16780: b'Center of Image Mass Y',
 16781: b'Center of Image Mass Y',
 16782: b'Center of Image Mass Y',
 16783: b'Center of Image Mass Y',
 16784: b'Center of Image Mass Z',
 16785: b'Center of Image Mass Z',
 16786: b'Center of Image Mass Z',
 16787: b'Center of

In [10]:
# np.savetxt('test.txt', list(out_stats_names.values()), fmt='%s')

In [11]:
# once we have the statistics names we can get numerical statistics values
# for each object id within the specified object
def get_stats_values(full_data_file: h5py.File, object_name: str) -> pd.DataFrame:
    """
    for a given object_name, extracts the statistics values for all object ids
    within the object

    Args:
        full_data_file (h5py._hl.files.File): full imaris file in h5py File format
        object_name (str): name of the object to get statistic names from

    Returns:
        pd.DataFrame: a pandas data frame that contains information about each object id
        where each object id has a stats id and associated stats value.
    """
    obj_specific_stats = full_data_file.get("Scene8").get("Content")[object_name][
        "StatisticsValue"
    ]
    obj_specific_stats = np.asarray(obj_specific_stats)
    return pd.DataFrame(obj_specific_stats)


out = get_stats_values(data, "MegaSurfaces0").drop("ID_Time", axis=1)
print(out)

ID_Object  ID_StatisticsType         Value
0               -1                 16  2.675000e+03
1               -1               1828  4.595600e+04
2               -1                768  4.595600e+04
3               -1               1827  1.607588e+07
4               -1              11260  3.436532e+06
...            ...                ...           ...
6452821      66616              16991  1.725892e+02
6452822      66867              16991  2.079986e+02
6452823      66875              16991  8.757520e+01
6452824      66920              16991  1.668989e+02
6452825      66924              16991  9.434139e+01

[6452826 rows x 3 columns]

In [18]:
out[out["ID_StatisticsType"] == 16991].sort_values("ID_Object")

,ID_Object,ID_StatisticsType,Value
403825,0,16991,187.485596
403826,2,16991,260.298279
403827,8,16991,521.410400
403828,12,16991,132.290390
403829,21,16991,2743.347168
...,...,...,...
6240403,70651,16991,108.746651
6350553,70652,16991,35.830444
6350558,70653,16991,102.009758
6350559,70654,16991,3.420808


In [12]:
extracted_stats = (
    out.groupby("ID_Object")[["ID_StatisticsType", "Value"]]
    .apply(lambda x: x.set_index("ID_StatisticsType").to_dict(orient="dict"))
    .to_dict()
)
extracted_stats = {
    k: v["Value"] for k, v in extracted_stats.items()
}  # clean up step for line above

In [13]:
def invert_stats_dict(stats_dict: dict = None):
    """
    creates a inverted_stats_dict --> [Statistics Type: Numeric Value]
    """

    # sort the dict
    sorted_dict_key = sorted(stats_dict.keys())
    stats_dict = {key: str(stats_dict[key]).strip("b")[1:-1] for key in sorted_dict_key}

    # create an empty dictionary
    storage = {}

    for key in stats_dict.keys():
        # if the word is not in the new storage dict
        if stats_dict[key] not in storage.keys():
            storage[stats_dict[key]] = key
        else:
            # get the value inside the key
            current_value = storage[stats_dict[key]]

            # if its a single value create a dict else create a dict
            if type(current_value) != dict:
                # then its the first value
                storage[stats_dict[key]] = {}
                storage[stats_dict[key]]["channel_1"] = current_value

                # current value
                storage[stats_dict[key]]["channel_2"] = key
            else:
                # get the length of the dict
                count = len(current_value.keys())
                # updated count
                count += 1
                # create new key
                new_key = f"channel_{count}"
                current_value[new_key] = key
                storage[stats_dict[key]] = current_value

    return storage


# we can directly use this to invert the dict
# this inverted dict can be used to create the final excel file

In [14]:
inverted_dict = invert_stats_dict(out_stats_names)
categories = [
    "Position X",
    "Position Y",
    "Position Z",
    "Intensity Mean_channel_1",
    "Intensity Mean_channel_2",
    "Intensity Mean_channel_3",
    "Intensity Mean_channel_4",
    "Intensity Mean_channel_5",
    "Volume",
]

In [15]:
# flatten dict
from collections.abc import MutableMapping


def flatten(d, parent_key="", sep="_"):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [16]:
inverted_flatten = flatten(inverted_dict)
# can we provide an editable list before anything starts so he can manually select which ones he wants
# that way the names match exactly because I control it.

# reverse dict again, key=num, value=name
final_dict = {v: k for k, v in inverted_flatten.items()}

# can we use the categories reqested to create a list of numbers we dont want?
del_list = [None] * (len(inverted_flatten) - len(categories))
counter = 0
for idx, (k, v) in enumerate(inverted_flatten.items()):
    if k not in categories and v != None:
        del_list[counter] = v
        counter += 1

In [28]:
final_dict[170]

'Intensity Mean_channel_1'

In [17]:
# map what the user wants to numeric values
user_request_numeric = [inverted_flatten[x] for x in categories]

# we can look at the final dict where key=num, value=name to filter
final_dict_copy = copy.deepcopy(final_dict)
for value in user_request_numeric:
    final_dict_copy.pop(value)

new_del_list = list(final_dict_copy.keys())

In [18]:
print(len(final_dict_copy.keys()), len(final_dict.keys()), len(user_request_numeric))

91 100 9

In [21]:
final_df = pd.DataFrame(extracted_stats).transpose()
f = final_df.drop(labels=del_list, axis=1)
print(f)

460       170        461       171        462           492  \
-1          NaN       NaN        NaN       NaN        NaN           NaN   
 2     1.864865  1.574324  34.121620  6.628378  47.337837  60797.417969   
 7     0.873950  1.193277  32.470589  4.731092   5.436975  60793.320312   
 10    0.939024  1.426829  30.585365  5.231707   5.085366  60812.757812   
 14    1.077029  1.112792  36.308117  5.674003  32.876205  60806.167969   
...         ...       ...        ...       ...        ...           ...   
 6627  1.775000  2.412500  32.974998  7.125000  47.662498  59255.476562   
 6629  1.607143  1.535714  32.446430  6.071429  38.875000  59191.093750   
 6630  1.903226  1.161290  35.048386  5.580645   8.483871  59170.410156   
 6632  0.850000  1.441667  32.658333  4.341667  32.666668  59209.675781   
 6633  1.579618  1.082803  33.668789  6.133758  25.585987  59166.628906   

                493        494          498  
-1              NaN        NaN          NaN  
 2     42978.500000   8.622030   702.604126  
 7     42925.894531  10.504700   562.719177  
 10    42901.535156  11.220044   347.759796  
 14    42916.218750   9.763873  3688.319336  
...             ...        ...          ...  
 6627  41322.378906  24.501162   350.664978  
 6629  41379.593750  25.501207   237.852676  
 6630  41399.046875  21.678446   288.769348  
 6632  41353.503906  21.967707   517.085388  
 6633  41391.062500  22.943762   744.258789  

[3246 rows x 9 columns]

In [23]:
final_df = pd.DataFrame(extracted_stats).transpose()
f = final_df.drop(labels=[None], axis=1)
print(f)

KeyError: '[None] not found in axis'

In [25]:
# clean up df by changing number to names and rearranging columns
columns = f.columns
print(columns)

new_names = {key: final_dict[key] for key in columns}
print(new_names)

# rename dict

Int64Index([460, 170, 461, 171, 462, 492, 493, 494, 498], dtype='int64')

{
    460: 'Intensity Mean_channel_3',
    170: 'Intensity Mean_channel_1',
    461: 'Intensity Mean_channel_4',
    171: 'Intensity Mean_channel_2',
    462: 'Intensity Mean_channel_5',
    492: 'Position X',
    493: 'Position Y',
    494: 'Position Z',
    498: 'Volume'
}

In [26]:
final_df = f.rename(new_names, axis=1)
print(final_df.head())

Intensity Mean_channel_3  Intensity Mean_channel_1  \
-1                        NaN                       NaN   
 2                   1.864865                  1.574324   
 7                   0.873950                  1.193277   
 10                  0.939024                  1.426829   
 14                  1.077029                  1.112792   

     Intensity Mean_channel_4  Intensity Mean_channel_2  \
-1                        NaN                       NaN   
 2                  34.121620                  6.628378   
 7                  32.470589                  4.731092   
 10                 30.585365                  5.231707   
 14                 36.308117                  5.674003   

     Intensity Mean_channel_5    Position X    Position Y  Position Z  \
-1                        NaN           NaN           NaN         NaN   
 2                  47.337837  60797.417969  42978.500000    8.622030   
 7                   5.436975  60793.320312  42925.894531   10.504700   
 10                  5.085366  60812.757812  42901.535156   11.220044   
 14                 32.876205  60806.167969  42916.218750    9.763873   

          Volume  
-1           NaN  
 2    702.604126  
 7    562.719177  
 10   347.759796  
 14  3688.319336

In [57]:
# create the final id column
final_df["ID"] = final_df.index


# rearrange
final_order = [
    "ID",
    "Position X",
    "Position Y",
    "Position Z",
    "Intensity Mean_channel_1",
    "Intensity Mean_channel_2",
    "Intensity Mean_channel_3",
    "Intensity Mean_channel_4",
    "Intensity Mean_channel_5",
]

final = final_df[final_order]
print(final)

ID   Position X   Position Y  Position Z  Intensity Mean_channel_1  \
-1         -1          NaN          NaN         NaN                       NaN   
 0          0  2189.359131  2202.812256   52.783691                 22.600000   
 1          1  2183.935791  2195.771240   47.824745                 20.820824   
 2          2  2178.945068  2182.046631   37.426094                 13.137931   
 3          3  2204.449463  2120.961670   28.069572                 13.305555   
...       ...          ...          ...         ...                       ...   
 27815  27815     1.185095    16.314802   46.986015                 13.260870   
 27816  27816     2.155134    22.085581   32.336147                 12.732000   
 27817  27817    10.448586     0.748731   60.644127                 19.453703   
 27818  27818    13.477354     9.190427   60.877293                 19.097952   
 27819  27819     2.357662     7.870090   59.777782                 22.074074   

        Intensity Mean_channel_2  Intensity Mean_channel_3  \
-1                           NaN                       NaN   
 0                     12.806896                  6.082758   
 1                      8.719129                  4.641646   
 2                      5.896552                  3.241379   
 3                     14.666667                  8.583333   
...                          ...                       ...   
 27815                  6.739130                  3.260870   
 27816                  4.512000                  2.368000   
 27817                 11.342592                  4.981482   
 27818                 10.386044                  3.898207   
 27819                  9.849537                  5.226852   

        Intensity Mean_channel_4  Intensity Mean_channel_5  
-1                           NaN                       NaN  
 0                      8.055172                  6.006896  
 1                      6.324455                  4.612591  
 2                      5.655172                  3.413793  
 3                     12.138889                  7.750000  
...                          ...                       ...  
 27815                  3.934783                  4.130435  
 27816                  3.368000                  4.312000  
 27817                  7.148148                  5.870370  
 27818                  4.567862                  5.507682  
 27819                  6.215278                  6.120370  

[26014 rows x 9 columns]